# Calling genotypes, prevalences and filtering data
This program analyzes tables of counts. In the prior "variant calling" step of the program, mutation counts were generated for each sample by annotating the VCF file generated by freebayes, using SNPEff to assign amino acid names to mutations.

The count tables are intended to be as "raw" as possible. In this part of the analysis we will:
1. apply various filters to the count tables, considering a mutation 'sampled' for a sample if it passes the filters for that sample.
2. choose the level of detail of geographic locations we care about from a metadata file
3. summarize the number of filtered samples at each location that had the mutation and the total number of filtered samples at each location, where the 'prevalence' of a mutation at a location is the first number divided by the second number
4. graph the prevalences and filtered sample sizes on a map
5. (optional) - apply additional filtering for re-evaluation of prevalence data.

In [7]:
# Edit
output_folder = 'output'
mutation_count_file = "input/new_tables/alternate_AA_table.csv"
mutation_coverage_file = "input/new_tables/coverage_AA_table.csv"
metadata_table = "input/new_tables/randomly_sampled_metadata.tsv"

# how many reads (or, more accurately, UMIs) do we need spanning the genomic coordinates of a mutation in order to
# consider the coordinates "sequenced" for a given sample? If UMI count is below min_coverage, the sample will be
# marked as "missing" for the mutation of interest.
min_coverage = 3

# If min_coverage is achieved in the sample, how many UMIs do we need that support the mutant (or alternate) allele
# in order to consider the alternate allele "present" in the sample? Otherwise if min_coverage is surpassed but
# min_count is not, the sample will be marked as matching the reference allele.
min_count = 1

# Is there a number that min_count/min_coverage needs to surpass before we consider an alternate allele "present" in a
# sample? Our usual answer is 'no' so we generally set this to '0' - but you may not want to count an alternate allele
# present if it's found in a very low fraction of the UMIs associated with a sample, even if the absolute number of UMIs
# associated with the alternate allele is above your threshold.
min_freq = 0

# edit mutations of interest (may want to come back to edit this after running the cell and checking on the observed missense variations)
mutations_of_interest=[
    'crt-Lys76Thr',
    'k13-Arg561His',
    'dhfr-ts-Ile164Leu',
    'dhps-Ala581Gly',
    'mdr1-Asn86Tyr'
]

# Don't Edit below
import sys
sys.path.append("src")
import background as bg
import importlib
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
importlib.reload(bg)
filtered_mutations = bg.filter_mutations(mutation_count_file)
bg.create_prevalences_input_table(
    mutations_of_interest,
    output_folder, 
    min_count, 
    min_coverage, 
    min_freq,
    mutation_count_file,
    mutation_coverage_file
)
print('observed missense mutations are:')
print(filtered_mutations)

importlib.reload(bg)
sample_column, summary_column = bg.generate_sample_summary_dropdowns(metadata_table)   
interact(
    bg.make_a_plot, 
    sample_col = sample_column, 
    summary_col = summary_column, 
    variant = mutations_of_interest, 
    country = bg.generate_country_dropdown()[0], 
    country_shortcuts = fixed(bg.generate_country_dropdown()[1]),
    output_folder = fixed(output_folder),
    metadata_table = fixed(metadata_table),
    mutations_of_interest = fixed(mutations_of_interest),
    zoom = '',
    center_lat = '',
    center_long = ''
)

observed missense mutations are:
['crt-Cys72Ser', 'crt-Val73Leu', 'crt-Met74Ile', 'crt-Asn75Glu', 'crt-Lys76Thr', 'crt-Thr93Ser', 'crt-His97Leu', 'crt-His97Tyr', 'crt-Cys101Phe', 'dhfr-ts-Ile164Leu', 'dhps-Ala581Gly', 'dhps-Ala613Ser', 'dhps-Ala613Thr', 'k13-Pro527His', 'k13-Asn537Asp', 'k13-Asn537Ile', 'k13-Gly538Val', 'k13-Arg539Thr', 'k13-Ile543Thr', 'k13-Pro553Leu', 'k13-Arg561His', 'k13-Val568Gly', 'k13-Pro574Leu', 'k13-Cys580Tyr', 'mdr1-Asn86Tyr']


interactive(children=(Dropdown(description='Sample:', options=('Region', 'Region_code', 'District', 'HF_name',…

<function background.make_a_plot(sample_col, summary_col, variant, country, country_shortcuts, output_folder, metadata_table, mutations_of_interest, zoom, center_lat, center_long)>

# Calulate Prevalences for Chosen Variants and Summary Columns

In [ ]:
# don't edit
importlib.reload(bg)
sample_column, summary_column = bg.generate_sample_summary_dropdowns(metadata_table)   
interact(
    bg.make_a_plot, 
    sample_col = sample_column, 
    summary_col = summary_column, 
    variant = mutations_of_interest, 
    country = bg.generate_country_dropdown()[0], 
    country_shortcuts = fixed(bg.generate_country_dropdown()[1]),
    output_folder = fixed(output_folder),
    metadata_table = fixed(metadata_table),
    mutations_of_interest = fixed(mutations_of_interest),
    zoom = '',
    center_lat = '',
    center_long = ''
)

## Static map of prevalences
Creates an SVG file of mapped mutations

In [ ]:
svg_country = bg.get_countries_from_geojson()
display(svg_country)

In [ ]:
# Edit

graph_states_provinces = True
latitude_range = [-1.6, 4.4]
longitude_range = [29, 35]
scale_factor=1

# additional labels to put on graph
labels = [
    ['Rwanda', -1.84, 30],
    ['Kenya', 0.93, 35.1],
    ['Tanzania', -1.75, 34.2],
    ['Ethiopia', 4.8, 37.4],
    ['S. Sudan', 4.16, 32.4],
    ['DRC', 1.5, 29]
]
annotation_font_size = 8

# title text
title_text = variant.value+' Prevalence'
title_size = 20

# Don't Edit
fig = bg.create_static_maps(
    variant.value,
    svg_country.value,
    graph_states_provinces,
    latitude_range,
    longitude_range,
    scale_factor,
    labels,
    annotation_font_size,
    title_text,
    title_size,
    output_folder,
    summary_column
)

fig.show()

## Bar Chart
Creates a Bar chart of mutation prevalences in the overall dataset

In [ ]:
# RUN
# This code generates a bar chart for your mutations of interest
def get_prevalences(prevalence_file, mutation_list):
	prevalence_dict={'mutations':[], 'prevalences':[], 'gene_name':[]}
	h_dict={}
	for line_number, line in enumerate(open(prevalence_file)):
		line=line.strip().split('\t')
		if line_number==0:
			for column_number, column in enumerate(line):
				h_dict[column]=column_number
		elif line[0]=='overall':
			for mutation in mutation_list:
				if mutation in h_dict:
					column=h_dict[mutation]
					prevalence=float(line[column].split(' ')[0])
					prevalence_dict['mutations'].append(mutation)
					prevalence_dict['prevalences'].append(prevalence)
					prevalence_dict['gene_name'].append(mutation.split('-')[0])
				else:
					print(mutation, f'is not found in {prevalence_file}!!')
	return prevalence_dict

def make_bar_chart(prevalence_dict, graph_type="bar chart of variants_of_interest", y_max=1.0):
	import pandas as pd
	import plotly.express as px
	df = pd.DataFrame(prevalence_dict)
	fig1=px.bar(df, x='mutations', y='prevalences', color='gene_name', text_auto=False)
	fig1['layout']['xaxis']['title']=graph_type
	fig1['layout']['yaxis']['title']='overall prevalence'
	fig1.update_layout(yaxis=dict(range=[0, y_max]))
	fig1.write_html(f'{graph_type}.html')
	fig1.write_image(f'{graph_type}.svg')
	fig1.write_image(f'{graph_type}.png')
	return fig1
chart_name='output/drug resistance mutations'
y_max=1.0

In [ ]:
# OPTIONAL USER INPUT
# User can optionally reset the variants_of_interest to graph, chart_name, and y_max values (e.g. by commenting in one
#of the blocks of lines below), otherwise the values will be pulled from the cells above

#variants_of_interest=['crt-Met74Ile', 'crt-Asn75Glu', 'crt-Lys76Thr','crt-Gln271Glu',
#'crt-Arg371Ile', 'dhfr-ts-Asn51Ile', 'dhfr-ts-Cys59Arg', 'dhfr-ts-Ser108Asn',
#'dhfr-ts-Ile164Leu', 'dhps-Ala437Gly', 'dhps-Lys540Glu', 'dhps-Ala581Gly',
#'mdr1-Asn86Tyr', 'mdr1-Tyr184Phe', 'mdr1-Asp650Asn', 'mdr1-Asn652Asp',
#'mdr1-Asp1246Tyr']
#chart_name='non-k13 mutations'
#y_max=1.0

#variants_of_interest=['k13-Lys108Glu', 'k13-His136Asn', 'k13-Lys189Asn', 'k13-Lys189Thr',
#'k13-Arg255Lys', 'k13-Leu258Met', 'k13-Gln271His', 'k13-Cys469Phe',
#'k13-Cys469Tyr', 'k13-Asn490Thr', 'k13-Arg561His', 'k13-Ala578Ser',
#'k13-Ala675Val']
#chart_name='k13 mutations'
#y_max=0.5


# NO USER INPUT BELOW
prevalence_dict=get_prevalences('output/prevalence_summary.tsv', variants_of_interest)
fig1=make_bar_chart(prevalence_dict, chart_name, y_max)
fig1.show()